## Notebook settings

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from scipy.stats import describe
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Conv1D, Input, Dropout, AvgPool1D, Reshape, Concatenate, Activation
from keras.activations import softmax 
from keras.initializers import Ones
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Init google drive
from google.colab import drive
from datetime import timedelta, date, datetime
# Plottool
from plotly import tools
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# Prophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
# IPython
from IPython.display import display
# Import math
import math
import os
import csv

drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'stock'

pd.options.display.max_columns = 12
pd.options.display.max_rows = 24

# disable warnings in Anaconda
warnings.filterwarnings('ignore')

# plots inisde jupyter notebook
%matplotlib inline

sns.set(style='darkgrid', palette='muted')
color_scheme = {
    'red': '#F1637A',
    'green': '#6ABB3E',
    'blue': '#3D8DEA',
    'black': '#000000'
}

# use svg for all plots within inline backend
%config InlineBackend.figure_format = 'svg'

# increase default plot size
rcParams['figure.figsize'] = 8, 6

Using TensorFlow backend.


Mounted at /content/gdrive


In [0]:
#For google colab, each time we use plotly, we have to call this function (IDK Why)
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

## Hãy load dữ liệu và vẽ một số biểu đồ


In [0]:
#This file have been preloaded
#df_org = pd.read_csv(f'{base_dir}/Nifty.csv', parse_dates=['Date'])
#df_org=df_org[['Date', 'Price']]
#df_org.rename(columns={'Date' : 'ds', 'Price' : 'y'}, inplace=True)

In [0]:
#Uncomment for gold data download
#pip install fix_yahoo_finance
import fix_yahoo_finance as yf
stock_name = '000001.SS' # SSE Composite Index
#stock_name= '^BSESN' #S&P BSE SENSEX
#stock_name = 'GLD'
df_org = yf.download(stock_name,start = "1990-01-01", end = "2016-12-31")
df_org.to_csv(f'{base_dir}/{stock_name}.csv')
#df_org = pd.read_csv(f'{base_dir}/{stock_name}.csv', parse_dates=['Date'])
df_org.reset_index(inplace=True)
df_org = df_org[['Date', 'Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']]

[*********************100%***********************]  1 of 1 downloaded


In [0]:
# File to save first results
save_fname = os.path.join(base_dir, '%s-%s.csv' % (stock_name, datetime.now().strftime('%d%m%Y-%H%M%S')))
of_connection = open(save_fname, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['stock_name', 'year', 'loss', 'params', 'iteration', 'windows_size', 'train_time'])
of_connection.close()



##Xây dựng bộ dữ liệu để train và xác minh

In [0]:
#df_org = df_org[:365]
df_org.tail(10)

,Date,Close,Open,High,Low,Adj Close,Volume
6575,2016-12-19,3118.084961,3120.696045,3125.283936,3110.082031,3118.084961,154200
6576,2016-12-20,3102.875977,3115.897949,3117.014893,3084.800049,3102.875977,157400
6577,2016-12-21,3137.429932,3107.239990,3140.250977,3107.239990,3137.429932,186700
6578,2016-12-22,3139.558105,3132.156006,3143.168945,3126.886963,3139.558105,167800
6579,2016-12-23,3110.154053,3134.928955,3138.404053,3103.748047,3110.154053,166200
6580,2016-12-26,3122.569092,3095.579102,3122.881104,3068.415039,3122.569092,0
6581,2016-12-27,3114.664063,3117.386963,3127.883057,3113.745117,3114.664063,141500
6582,2016-12-28,3102.236084,3113.767090,3118.781982,3094.549072,3102.236084,135700
6583,2016-12-29,3096.096924,3095.844971,3111.799072,3087.343994,3096.096924,132600
6584,2016-12-30,3103.636963,3097.344971,3108.839111,3089.989014,3103.636963,133300


In [0]:
#Declare const
input_col = ['Close', 'Open', 'High', 'Low', 'Adj Close', 'Volume']
output_col = ['Close']
time_col = ['Date']

#Số đầu vào của mỗi hàng
input_dim = len(input_col)
#Số đầu ra của mỗi hàng
output_dim = len(output_col)

prediction_size = 1
#Hiễn thị 5 cột đầu tiên để kiểm tra
sample_display_test_size = 5
#Số lần chạy bayers mỗi lần
max_evals = 10


In [0]:
#declare model
def softMaxAxis1(x):
    return softmax(x,axis=1)


def get_model(input_dim, window_size, output_dim):
  complex_model = Sequential()
  complex_model.add(LSTM(units=100, input_shape=(window_size, input_dim), return_sequences=True, kernel_initializer=Ones()))
  complex_model.add(Dropout(rate=0.2))
  complex_model.add(LSTM(units=100, return_sequences=True))
  complex_model.add(Dropout(rate=0.2))
  complex_model.add(LSTM(units=100, return_sequences=True))
  complex_model.add(Dropout(rate=0.2))
  complex_model.add(LSTM(units=100, return_sequences=True))
  complex_model.add(Dropout(rate=0.2))
  complex_model.add(LSTM(output_dim, activation=softMaxAxis1))
  complex_model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
 
  return complex_model

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_square_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean((y_true - y_pred) / y_true)
  
def relative_root_mean_square_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    res = (y_true - y_pred) / y_true
    res = np.power(res, 2)
    res = np.mean(res)
    res = math.sqrt(res)
    
    return res

  
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('Date').join(historical.set_index('Date'))
  


#reprocessing data
def next_window(df, i, windows_size, prediction_size):
    '''Generates the next data window from the given index location i'''
    window = df[i : i + windows_size + prediction_size]
    x = window[input_col][:-prediction_size]
    y = window[output_col][-prediction_size:]   
    y_time = window[time_col][-prediction_size:]
    return x, y, y_time

def preprocessing_data(df, windows_size, prediction_size):
    '''
    Create x, y train data windows
    Warning: batch method, not generative, make sure you have enough memory to
    load data, otherwise use generate_training_window() method.
    '''
    data_x = []
    data_y = []
    data_y_time = []
    for i in range(len(df) - windows_size - prediction_size):
        x, y, y_time = next_window(df, i, windows_size, prediction_size)
        data_x.append(x.values)
        data_y.append(y.values)
        data_y_time.append(y_time)
    
    time = pd.concat(data_y_time)
    
    return np.array(data_x), np.array(data_y), time.values

def split_train_test_data(X, y):
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=False)

  return X_train, y_train, X_valid, y_valid

#Trainning model
def train_model(model, X_train, y_train, X_valid, y_valid):
    callbacks = [
            EarlyStopping(monitor='val_loss', patience=50)
        ]
    history = model.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=10000,
    validation_data=(X_valid, y_valid),
    verbose=2,
    callbacks=callbacks,
    shuffle=False)
    
    return history

def test_model(model, test_data, window_size, prediction_size):

  X, y, time = preprocessing_data(test_data, window_size, prediction_size)
  y = y.reshape((y.shape[0], y.shape[1]))

  y_pred  = model.predict(X)

  y_pred = np.repeat(y_pred, input_dim, axis=1)
  y_pred = scaler.inverse_transform(y_pred)[:, [0]]
  y_pred = pd.Series(y_pred.flatten())

  df_test_result = pd.DataFrame(time, columns=['Date'])
  df_test_result['Prediction'] = y_pred
  df_test_result.set_index('Date', inplace=True)
 
  return df_test_result

In [0]:
def plot_test_result(test_result):

  # Plotly
  trace0 = go.Scatter(
      x = test_result.index,
      y = test_result['Close'],
      name = 'Thực tế',
      line = dict(
          color = ('#5042f4'),
          width = 2)
  )

  trace1 = go.Scatter(
      x = test_result.index,
      y = test_result['Prediction'],
      name = 'Dự đoán',
      line = dict(
          color = ('#005b4e'),
          width = 2,
          dash = 'dot'
      ) # dash options include 'dash', 'dot', and 'dashdot'
  )

  data = [trace0, trace1]


  # Edit the layout
  layout = dict(title = 'Biểu đồ dự đoán',
                xaxis = dict(title = 'Date'),
                yaxis = dict(title = 'Price'),
                paper_bgcolor='#FFF9F5',
                plot_bgcolor='#FFF9F5'
                )

  fig = go.Figure(data=data, layout=layout)
  fig.show()

In [0]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import tpe
from hyperopt import fmin
from hyperopt import STATUS_OK
from timeit import default_timer as timer

def objective(params):
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Make sure windows_size is int
    windows_size = int(params['windows_size'])
    print(f'Window size is {windows_size}')

    complex_model = get_model(input_dim, windows_size, output_dim)

    start = timer()

    # Handle data
    df.describe()
    X, y, time = preprocessing_data(df, windows_size, prediction_size)

    # Reshape data
    y = y.reshape((y.shape[0], y.shape[1]))

    X_train, y_train, X_valid, y_valid = split_train_test_data(X, y)
    
    # Perform n_train
    history = train_model(complex_model, X_train, y_train, X_valid, y_valid)
    
    run_time = timer() - start
    
    
    # Test generated loss
    test_result = test_model(complex_model, df, windows_size, prediction_size)    
    test_result = test_result.join(df_org.set_index('Date'))

    mae = mean_absolute_error(test_result['Close'], test_result['Prediction'])
    mse = mean_squared_error(test_result['Close'], test_result['Prediction'])
    mape = mean_absolute_percentage_error(test_result['Close'], test_result['Prediction'])
    rrmse = relative_root_mean_square_error(test_result['Close'], test_result['Prediction'])

    print(f'{stock_name} prediction for {prediction_size} day ahead')
    print(f'MAE = {mae}')
    print(f'MSE = {mse}')
    print(f'MAPE = {mape}')
    print(f'RRMSE = {rrmse}')

    display(test_result.head(sample_display_test_size))
    loss = mae

    #write row
    of_connection = open(save_fname, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([stock_name, year, loss, params, ITERATION, windows_size  ,run_time])
    
    # Dictionary with information for evaluation` v
    return {'loss': loss, 'params': params, 'iteration': ITERATION, 'test_result': test_result,
            'train_time': run_time, 'status': STATUS_OK}


# Run evals with the tpe algorithm
#bayes_best = fmin(fn=objective, space=param_grid, 
#                algo=bayes_algo, trials=bayes_trials, 
#                max_evals=10)
#
#print(bayes_best)

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [0]:
start_year = df_org['Date'].values[:1][0]
start_year = pd.to_datetime(start_year).year

end_year = df_org['Date'].values[-1:][0]
end_year = pd.to_datetime(end_year).year

windows_size_best = []
# Global variable
global  ITERATION
test_result = 0
for year in range(start_year, end_year + 1):
  df = df_org[df_org['Date'].dt.year == year]

  display(df.head())
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_cols = scaler.fit_transform(df[input_col])
  df[input_col] = scaled_cols

  # Hyperparameter grid
  param_grid = {
      'windows_size': hp.choice('windows_size', np.arange(1, 8, dtype=int))
  }

  bayes_trials = Trials()

  # Create the algorithm
  bayes_algo = tpe.suggest
  
  ITERATION = 0

  bayes_best = fmin(fn=objective, space=param_grid, 
                 algo=bayes_algo, trials=bayes_trials, 
                 max_evals=max_evals)
  
  windows_size_best.append([year, bayes_best])
  print(f'LOG:RESULT Bayer best for {year} = {bayes_best}')

  

,Date,Close,Open,High,Low,Adj Close,Volume
0,1990-12-19,99.980003,99.980003,99.980003,99.980003,99.980003,0
1,1990-12-20,104.389999,104.389999,104.389999,104.389999,104.389999,0
2,1990-12-21,109.129997,109.129997,109.129997,109.129997,109.129997,0
3,1990-12-24,114.550003,114.550003,114.550003,114.550003,114.550003,0
4,1990-12-25,120.250000,120.250000,120.250000,120.250000,120.250000,0


  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.002977 seconds
INFO:hyperopt.tpe:TPE using 0 trials


Window size is 6
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

Window size is 4
Train on 205 samples, validate on 52 samples
Epoch 1/100
 - 73s - loss: 0.3528 - acc: 0.0000e+00 - val_loss: 0.1056 - val_acc: 0.0192

Epoch 2/100
 - 0s - loss: 0.3385 - acc: 0.0000e+00 - val_loss: 0.1231 - val_acc: 0.0192

Epoch 3/100
 - 0s - loss: 0.3217 - acc: 0.0000e+00 - val_loss: 0.1537 - val_acc: 0.0192

Epoch 4/100
 - 0s - loss: 0.3007 - acc: 0.0000e+00 - val_loss: 0.1982 - val_acc: 0.0000e+00

Epoch 5/100
 - 0s - loss: 0.2783 - acc: 0.0000e+00 - val_loss: 0.2564 - val_acc: 0.0000e+00

Epoch 6/100
 - 0s - loss: 0.2586 - acc: 0.0000e+00 - val_loss: 0.3277 - val_acc: 0.0000e+00

Epoch 7/100
 - 0s - loss: 0.2538 - acc: 0.0000e+00 - val_loss: 0.3809 - val_acc: 0.0000e+00

Epoch 8/100
 - 0s - loss: 0.2553 - acc: 0.0000e+00 - val_loss: 0.4034 - val_acc: 0.0000e+00

Epoch 9/100
 - 0s - loss: 0.2594 - acc: 0.0000e+00 - val_loss: 0.4008 - val_acc: 0.0000e+00

Epoch 10/100
 - 0s - loss: 0.2524 - acc: 0.0000e+00 - val_loss: 0.3828 - val_acc: 0.0000e+00

Epoch 11/100
 - 0s

,Prediction,Close,Open,High,Low,Adj Close,Volume
Date,,,,,,,
1996-01-05,533.326660,536.369995,536.369995,536.369995,536.369995,536.369995,0
1996-01-08,539.490051,539.169983,539.169983,539.169983,539.169983,539.169983,0
1996-01-09,540.661926,558.619995,558.619995,558.619995,558.619995,558.619995,0
1996-01-10,532.143921,547.619995,547.619995,547.619995,547.619995,547.619995,0
1996-01-11,539.894165,545.640015,545.640015,545.640015,545.640015,545.640015,0


 30%|███       | 3/10 [05:39<13:15, 113.71s/it, best loss: 38.63835485692789] 

INFO:hyperopt.tpe:tpe_transform took 0.002271 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss 38.638355


Window size is 4
Train on 205 samples, validate on 52 samples
Epoch 1/100
 - 74s - loss: 0.2507 - acc: 0.0049 - val_loss: 0.1909 - val_acc: 0.0000e+00

Epoch 2/100
 - 0s - loss: 0.2399 - acc: 0.0049 - val_loss: 0.2203 - val_acc: 0.0000e+00

Epoch 3/100
 - 0s - loss: 0.2306 - acc: 0.0049 - val_loss: 0.2549 - val_acc: 0.0000e+00

Epoch 4/100
 - 0s - loss: 0.2217 - acc: 0.0049 - val_loss: 0.2963 - val_acc: 0.0000e+00

Epoch 5/100
 - 0s - loss: 0.2158 - acc: 0.0049 - val_loss: 0.3281 - val_acc: 0.0000e+00

Epoch 6/100
 - 0s - loss: 0.2170 - acc: 0.0049 - val_loss: 0.3393 - val_acc: 0.0000e+00

Epoch 7/100
 - 0s - loss: 0.2141 - acc: 0.0049 - val_loss: 0.3351 - val_acc: 0.0000e+00

Epoch 8/100
 - 0s - loss: 0.2126 - acc: 0.0049 - val_loss: 0.3224 - val_acc: 0.0000e+00

Epoch 9/100
 - 0s - loss: 0.2082 - acc: 0.0049 - val_loss: 0.3079 - val_acc: 0.0000e+00

Epoch 10/100
 - 0s - loss: 0.2044 - acc: 0.0049 - val_loss: 0.2936 - val_acc: 0.0000e+00

Epoch 11/100
 - 0s - loss: 0.2027 - acc: 0.004

In [0]:
windows_size_best

In [0]:
plot_test_result(bayes_trials.results[-1]['test_result'])